In [9]:
import os
#os.environ['CUDA_HOME'] = "/usr/local/cuda-12.4"
print(os.environ['CUDA_HOME'])

/usr/local/cuda-12.4


In [10]:
# %pip install -r requirements.txt

In [11]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-vector-stores-postgres
%pip install llama-index-embeddings-huggingface
#%pip install llama-index-llms-llama-cpp

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
# This may need to be run outside of the notebook
#!CUDACXX=/usr/local/cuda-12.4/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir --force-reinstall --upgrade

In [13]:
#cpu
#%env CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"
#cuda
#%env CMAKE_ARGS="-DLLAMA_CUBLAS=on" 
#%env FORCE_CMAKE=1
#%env CUDAFLAGS="-arch=all -lcublas"
#%pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir --verbose

In [14]:
%pip install llama-index-llms-llama-cpp

Note: you may need to restart the kernel to use updated packages.


In [15]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [16]:
#from huggingface_hub import hf_hub_download
#hf_hub_download(repo_id='TheBloke/Mistral-7B-v0.1-GGUF', filename='mistral-7b-v0.1.Q4_K_M.gguf', local_dir='./models', local_dir_use_symlinks=False)

In [17]:
from llama_index.llms.llama_cpp import LlamaCPP

In [18]:
# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"
model_url = "https://huggingface.co/TheBloke/Mistral-7B-v0.1-GGUF/resolve/main/mistral-7b-v0.1.Q4_K_M.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=None,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path='models/mistral-7b-v0.1.Q4_K_M.gguf',
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 256},
    verbose=True,
)


llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from models/mistral-7b-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_

In [19]:
%pip install psycopg2-binary pgvector asyncpg "sqlalchemy[asyncio]" greenlet

Note: you may need to restart the kernel to use updated packages.


In [20]:
# import psycopg2

# db_name = "vector_db"
# host = "localhost"
# password = "postgres"
# port = "5432"
# user = "postgres"
# # conn = psycopg2.connect(connection_string)
# conn = psycopg2.connect(
#     dbname=postgres,
#     host=host,
#     password=password,
#     port=port,
#     user=user,
# )
# conn.autocommit = True

# with conn.cursor() as c:
#     c.execute(f"DROP DATABASE IF EXISTS {db_name}")
#     c.execute(f"CREATE DATABASE {db_name}")


In [21]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

db_name = "vectordb"
# host = "db"
host = "localhost"
password = "testpwd"
port = "5432"
user = "testuser"
vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

In [22]:
#%mkdir data
#%wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

In [23]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

In [24]:
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))



In [25]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [26]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding


In [27]:
vector_store.add(nodes)

['a89fe542-54cd-431e-b6cd-aab78d15fc36',
 '9dabe2fc-29c8-4f1e-982e-0b0ef0418298',
 '663096c1-d791-4e29-b56c-c39759e72d9f',
 '3ed2d0ab-48d2-4b99-81ad-bd67e2188911',
 '281e82c9-b4ab-4be2-9c75-6228b9c747ce',
 '9aa1fbe5-856a-400c-8584-250e2ac7582c',
 'e4a57f78-c7ab-43f3-9440-bb34bf972c43',
 'e5fe5fce-c2a6-4876-90cf-1da4fa15aa10',
 '021590b9-b6bd-4da8-a5bb-7e76d26b689f',
 '75197d37-4c27-4f00-b359-ac14aa08c6d5',
 '1b736572-0710-433b-81f5-5a5abbe8ba2c',
 '1cf38ee2-fe7d-4eca-bbd3-4d303e421112',
 'cca8c56d-d03b-4e2d-a269-80e134f03507',
 '44b5a5e7-93c2-4b7b-8d8a-211779528b6e',
 'a6e73a64-bf1d-4045-ac4c-fda4c801b095',
 '1e337c7f-dff2-49e6-b23a-5a44b0c488af',
 '5035b960-b5af-4bc4-b618-59c8781c2ea8',
 '3b01e02e-0034-4f86-985f-b23a157420ca',
 '9d960203-038f-44e7-8e46-7c907479bc7a',
 'a7f0eb36-dbc7-4f71-9a51-5cd2c94425b9',
 'f654eb3f-6d71-429b-89a1-88de25e1964c',
 '59520dd9-0be2-4d44-b656-a8e73783e8c1',
 '921004f9-244e-4a9a-9d78-38886adb4b1c',
 '7d9f9760-a74d-45f4-a0f0-97099b730c8f',
 'de1b336b-140d-

In [28]:
query_str = "Can you tell me about the key concepts for safety finetuning"

In [29]:
query_embedding = embed_model.get_query_embedding(query_str)

In [30]:
from llama_index.core.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [31]:
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())


TruthfulQA ↑
ToxiGen ↓
MPT
7B
29.13
22.32
30B
35.25
22.61
Falcon
7B
25.95
14.53
40B
40.39
23.44
Llama 1
7B
27.42
23.00
13B
41.74
23.08
33B
44.19
22.57
65B
48.71
21.77
Llama 2
7B
33.29
21.25
13B
41.86
26.10
34B
43.45
21.19
70B
50.18
24.60
Table 11: Evaluation of pretrained LLMs on automatic safety benchmarks. For TruthfulQA, we present the
percentage of generations that are both truthful and informative (the higher the better). For ToxiGen, we
present the percentage of toxic generations (the smaller, the better).
Benchmarks give a summary view of model capabilities and behaviors that allow us to understand general
patterns in the model, but they do not provide a fully comprehensive view of the impact the model may have
on people or real-world outcomes; that would require study of end-to-end product deployments. Further
testing and mitigation should be done to understand bias and other social issues for the specific context
in which a system may be deployed. For this, it may be necessary

In [32]:
from llama_index.core.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))


In [33]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [34]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

In [35]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm, streaming=True)

In [42]:
query_str = "How does Llama 2 perform compared to other open-source models?"

response = query_engine.query(query_str)
response.print_response_stream()

Llama.generate: prefix-match hit


1. Llama 2 outperforms all open-source models on most benchmarks. 2. Llama 2 is close to GPT-3.5 (OpenAI, 2023) on MMLU and GSM8K, but there is a significant gap on coding benchmarks. 3. Llama 2 results are on par or better than PaLM (540B) (Chowdhery et al., 2022) on almost all benchmarks. 4. There is still a large gap in performance between Llama 2 and GPT-4 and PaLM-2-L.


llama_print_timings:        load time =     527.29 ms
llama_print_timings:      sample time =      22.95 ms /   143 runs   (    0.16 ms per token,  6230.39 tokens per second)
llama_print_timings: prompt eval time =     525.03 ms /   730 tokens (    0.72 ms per token,  1390.41 tokens per second)
llama_print_timings:        eval time =    2875.55 ms /   142 runs   (   20.25 ms per token,    49.38 tokens per second)
llama_print_timings:       total time =    3751.69 ms /   872 tokens


In [41]:
print(str(response))


7B, 13B, and 70B


In [38]:
print(response.source_nodes[0].get_content())


Additionally, Llama 2 70B model outperforms all open-source models.
In addition to open-source models, we also compare Llama 2 70B results to closed-source models. As shown
in Table 4, Llama 2 70B is close to GPT-3.5 (OpenAI, 2023) on MMLU and GSM8K, but there is a significant
gap on coding benchmarks. Llama 2 70B results are on par or better than PaLM (540B) (Chowdhery et al.,
2022) on almost all benchmarks. There is still a large gap in performance between Llama 2 70B and GPT-4
and PaLM-2-L.
We also analysed the potential data contamination and share the details in Section A.6.
Benchmark (shots)
GPT-3.5
GPT-4
PaLM
PaLM-2-L
Llama 2
MMLU (5-shot)
70.0
86.4
69.3
78.3
68.9
TriviaQA (1-shot)
–
–
81.4
86.1
85.0
Natural Questions (1-shot)
–
–
29.3
37.5
33.0
GSM8K (8-shot)
57.1
92.0
56.5
80.7
56.8
HumanEval (0-shot)
48.1
67.0
26.2
–
29.9
BIG-Bench Hard (3-shot)
–
–
52.3
65.7
51.2
Table 4: Comparison to closed-source models on academic benchmarks. Results for GPT-3.5 and GPT-4
are from OpenAI

In [39]:
query_str = "What is the parameter count of a Llama 2 model?"
response = query_engine.query(query_str)
response.print_response_stream()

Llama.generate: prefix-match hit


7B, 13B, and 70B


llama_print_timings:        load time =     527.29 ms
llama_print_timings:      sample time =       2.19 ms /    14 runs   (    0.16 ms per token,  6381.04 tokens per second)
llama_print_timings: prompt eval time =     587.98 ms /   997 tokens (    0.59 ms per token,  1695.64 tokens per second)
llama_print_timings:        eval time =     244.22 ms /    13 runs   (   18.79 ms per token,    53.23 tokens per second)
llama_print_timings:       total time =     864.54 ms /  1010 tokens


In [40]:
print(str(response))


7B, 13B, and 70B
